In [114]:
#! /usr/bin/env python
# -*- coding: utf-8 -*- 
#! python3 
import matplotlib.pyplot as plt # plt 用于显示图片
import matplotlib.image as mpimg # mpimg 用于读取图片
import numpy as np
import copy
import cv2
import PIL.Image as Image
import os
import pandas as pd

%matplotlib inline

plt.style.use("ggplot")

In [221]:

grid_height=5
grid_width=5

num_level=6

Havg=147
Hblack=0
Hwhite=255

Hbright=[]
Hdark=[]

for i in range(0,num_level):
    Hdark.append(round((Havg-Hblack)*i/num_level+Hblack))
    Hbright.append(Hwhite-Hdark[i])
print(Hdark,Hbright)

def generate_display_grid():
    length=100
    images=[]
    for l in range(num_level):
        grid_image=np.ones((length,length+2))
        for i in range(0,length,grid_height):
            for j in range(0,length,grid_width):
                if (i/grid_height+j/grid_width)%2==1:
                    for ii in range(i,min(length,i+grid_height)):
                        for jj in range(j,min(length,j+grid_width)):
                            grid_image[ii][jj]= Hdark[l]/255 
                else:
                    for ii in range(i,min(length,i+grid_height)):
                        for jj in range(j,min(length,j+grid_width)):
                            grid_image[ii][jj]=Havg/255
#         cv2.imshow('Test',grid_image) # 这里展示时需小于等于1.0
#         cv2.waitKey()
        grid_image=grid_image*255    # 保存图片时需取0-255
        images.append(grid_image)
        cv2.imwrite('ReproductionImages/HidenImage/DarkGrids/{}.png'.format(str(l+1)), grid_image)
#         cv2.imwrite('ReproductionImages/HidenImage/BrightGrids/{}.png'.format(str(l+1)), grid_image)
    images=np.concatenate(np.array(images),axis=1)
    cv2.imshow('Test',images/255) # 这里展示时需小于等于1.0
    cv2.waitKey()
    cv2.imwrite('ReproductionImages/HidenImage/DarkGrids/dark_partition_grid.png', images)
    return images
        
images=generate_display_grid()
print(type(images))
print(images.shape)

[0, 24, 49, 74, 98, 122] [255, 231, 206, 181, 157, 133]
<class 'numpy.ndarray'>
(100, 612)


In [222]:
data = cv2.imread('pictures/HidenImage/OriginalPerson'+str(1)+'.png')#,cv2.IMREAD_GRAYSCALE)
image = cv2.cvtColor(data,cv2.COLOR_BGR2GRAY)
# cv2.imshow('Test',data)
# cv2.waitKey()

h=image.shape[0]
w=image.shape[1]

def paint(image,x,y,grid_height,grid_width,h,w,c):
    for i in range(x,min(h,x+grid_height)):
        for j in range(y,min(w,y+grid_width)):
            image[i][j]=c
            
def average(image,x,y,grid_height,grid_width,h,w):
    sum=0.0
    count=(min(h,x+grid_height)-x)*(min(w,y+grid_width)-y)
    for i in range(x,min(h,x+grid_height)):
        for j in range(y,min(w,y+grid_width)):
            sum+=image[i][j]
    return sum/count      

hdark=Hdark.copy()
hdark.append(Havg)
# hdark=[a*255/Havg for a in hdark]

for i in range(0,h,grid_height):
    for j in range(0,w,grid_width):
        if (i/grid_height+j/grid_width)%2==1:
            paint(image,i,j,grid_height,grid_width,h,w,Havg)
        else:

            for k in range(0,num_level):
#                 if image[i][j]>=2*hdark[k] and image[i][j]<2*hdark[k+1]:
#                 if image[i][j]>=255*hdark[k]/Havg and image[i][j]<255*hdark[k+1]/Havg:
                avg=average(image,i,j,grid_height,grid_width,h,w)
                if avg>=2*hdark[k] and avg<2*hdark[k+1]:
#                 if avg>=255*hdark[k]/Havg and avg<255*hdark[k+1]/Havg:
                    paint(image,i,j,grid_height,grid_width,h,w,hdark[k])
            
#             if image[i][j]<Havg:
#                 hdark=Hdark
#                 hdark.append(Havg)
#                 for k in range(0,num_level):
#                     if image[i][j]>=hdark[k] and image[i][j]<hdark[k+1]:
#                         paint(image,i,j,grid_height,grid_width,h,w,hdark[k])
#             else:
#                 hbright=Hbright
#                 hbright.append(Havg)
#                 for k in range(0,num_level):
#                     if image[i][j]<=hbright[k] and image[i][j]>hbright[k+1]:
#                         paint(image,i,j,grid_height,grid_width,h,w,hbright[k])
            
                    
cv2.imshow('Test',image)
cv2.waitKey()
cv2.imwrite('ReproductionImages/HidenImage/hiden_person.png', image)

True

In [223]:
data = cv2.imread('pictures/HidenImage/OriginalPerson'+str(1)+'.png')#,cv2.IMREAD_GRAYSCALE)
image = cv2.cvtColor(data,cv2.COLOR_BGR2GRAY)
# cv2.imshow('Test',data)
# cv2.waitKey()

h=image.shape[0]
w=image.shape[1]
hdark=Hdark.copy()
hdark.append(Havg)
images=[]
for l in range(num_level):
    fig=image.copy()
    for i in range(h):
        for j in range(w): 
#             if image[i][j]>=2*hdark[l] and image[i][j]<2*hdark[l+1]:
#             if image[i][j]>=255*hdark[l]/Havg and image[i][j]<255*hdark[l+1]/Havg:
            avg=average(image,i,j,grid_height,grid_width,h,w)            
#             if avg>=255*hdark[l]/Havg and avg<255*hdark[l+1]/Havg:
            if avg>=2*hdark[l] and avg<2*hdark[l+1]:
                fig[i][j]=255
            else:
                fig[i][j]=0
    images.append(fig)
images=np.concatenate(np.array(images),axis=1)
# cv2.imshow('Test',images/255) # 这里展示时需小于等于1.0
# cv2.waitKey()
cv2.imwrite('ReproductionImages/HidenImage/level_mark.png', images)

True

In [32]:
data = cv2.imread('pictures/HidenImage/'+str(1)+'.png')#,cv2.IMREAD_GRAYSCALE)
# image = cv2.cvtColor(data,cv2.COLOR_BGR2GRAY)
# cv2.imshow('Test',data)
# cv2.waitKey()
print(type(data))

h=image.shape[0]
w=image.shape[1]

a=image.copy()
print(image[0][50])
print(a[0][50])
a[0][50]=223
print(image[0][50])
print(a[0][50])

<class 'numpy.ndarray'>
90
90
90
223
